# Logistic Regressin custom reference category

When reporting odds ratios for categorical variables in a logistic regression model, one category is chosen as the "reference" and odd ratios are reported for others, relative to this reference.

What if we want to report numbers using another category as the base case?

In [22]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Logistic Regression

Consider this excellent writeup of the statistical analysis of an admissions data set:

https://stats.idre.ucla.edu/r/dae/logit-regression/

cf.  also https://stats.oarc.ucla.edu/stata/dae/logistic-regression/

In [2]:
# data = whether students got admitted (admit=1) or not (admit=0) based on their gre and gpa scores, and the rank of their instutution
# raw_data = pd.read_csv('https://stats.idre.ucla.edu/stat/data/binary.csv')
raw_data = pd.read_csv('./data/binary.csv')
raw_data

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
...,...,...,...,...
395,0,620,4.00,2
396,0,560,3.04,3
397,0,460,2.63,2
398,0,700,3.65,2


In [3]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   admit   400 non-null    int64  
 1   gre     400 non-null    int64  
 2   gpa     400 non-null    float64
 3   rank    400 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 12.6 KB


In [4]:
 raw_data.describe()

,admit,gre,gpa,rank
count,400.000000,400.000000,400.000000,400.00000
mean,0.317500,587.700000,3.389900,2.48500
std,0.466087,115.516536,0.380567,0.94446
min,0.000000,220.000000,2.260000,1.00000
25%,0.000000,520.000000,3.130000,2.00000
50%,0.000000,580.000000,3.395000,2.00000
75%,1.000000,660.000000,3.670000,3.00000
max,1.000000,800.000000,4.000000,4.00000


In [5]:
# convert rank to categorical
# via https://stackoverflow.com/a/39092877
mydata = raw_data.copy()
mydata['rank'] = pd.Categorical(mydata['rank'])

In [6]:
mydata_crosstab = pd.crosstab(
    mydata['admit'],
    mydata['rank'], 
    margins = False
)
mydata_crosstab

rank,1,2,3,4
admit,,,,
0,28,97,93,55
1,33,54,28,12


In [7]:
mydata.dtypes

admit       int64
gre         int64
gpa       float64
rank     category
dtype: object

In [25]:
mylogit = smf.logit(
    'admit ~ gre + gpa + rank',
    data=mydata
).fit()

mylogit

Optimization terminated successfully.
         Current function value: 0.573147
         Iterations 6


In [26]:
mylogit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  400
Model:                          Logit   Df Residuals:                      394
Method:                           MLE   Df Model:                            5
Date:                Sun, 26 May 2024   Pseudo R-squ.:                 0.08292
Time:                        09:38:50   Log-Likelihood:                -229.26
converged:                       True   LL-Null:                       -249.99
Covariance Type:            nonrobust   LLR p-value:                 7.578e-08
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.9900      1.140     -3.500      0.000      -6.224      -1.756
rank[T.2]     -0.6754      0.316     -2.134      0.033      -1.296      -0.055
rank[T.3]     -1.3402      0.345     -3.881      0.000      -2.017      -0.663
rank[T.4]     -1.5515      0.418     -3.713      0.000      -2.370      -0.733
gre            0.0023      0.001      2.070      0.038       0.000       0.004
gpa            0.8040      0.332      2.423      0.015       0.154       1.454
==============================================================================
"""

In [27]:
mylogit.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Method:           MLE       
Dependent Variable: admit            Pseudo R-squared: 0.083     
Date:               2024-05-26 09:38 AIC:              470.5175  
No. Observations:   400              BIC:              494.4663  
Df Model:           5                Log-Likelihood:   -229.26   
Df Residuals:       394              LL-Null:          -249.99   
Converged:          1.0000           LLR p-value:      7.5782e-08
No. Iterations:     6.0000           Scale:            1.0000    
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
Intercept     -3.9900    1.1400  -3.5001  0.0005  -6.2242  -1.7557
rank[T.2]     -0.6754    0.3165  -2.1342  0.0328  -1.2958  -0.0551
rank[T.3]     -1.3402    0.3453  -3.8812  0.0001  -2.0170  -0.6634
rank[T.4]     -1.5515    0.4178  -3.7131  0.0002  -2.3704  -0.7325
gre            0.0023    0.0011   2.0699  0.0385   0.0001   0.0044
gpa            0.8040    0.3318   2.4231  0.0154   0.1537   1.4544
=================================================================

"""

In [28]:
# mylogit.__dict__

In [29]:
mylogit.params

Intercept   -3.989979
rank[T.2]   -0.675443
rank[T.3]   -1.340204
rank[T.4]   -1.551464
gre          0.002264
gpa          0.804038
dtype: float64

The above model uses the rank=1 as the reference category an the log odds reported are with respect to this catrgory

log(accept|rank=1)/log(accept|rank=2) = rank[T.2] 	-0.675443

etc. for others
rank[T.3] 	-1.340204
rank[T.4] 	-1.551464



## Statement of the problem

How can we obtain the log odds with respect to another reference category, e.g. rank=2 

In [13]:

# Option one: custom function that permutes categories to put rank 2 as reference
# https://www.statsmodels.org/stable/example_formulas.html#functions

In [14]:
# Option 2: order categoricals so rank=2 comes first in the list at creation time
mydata2 = raw_data.copy()
mydata2['rank'] = pd.Categorical(mydata2['rank'], categories=[2,1,3,4])

In [15]:
mylogit2 = smf.mnlogit(
    'admit ~ gre + gpa + rank',
    data=mydata2
).fit()

mylogit2.summary()

Optimization terminated successfully.
         Current function value: 0.573147
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                  admit   No. Observations:                  400
Model:                        MNLogit   Df Residuals:                      394
Method:                           MLE   Df Model:                            5
Date:                Fri, 24 May 2024   Pseudo R-squ.:                 0.08292
Time:                        12:26:27   Log-Likelihood:                -229.26
converged:                       True   LL-Null:                       -249.99
Covariance Type:            nonrobust   LLR p-value:                 7.578e-08
==============================================================================
   admit=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.6654      1.109     -4.205      0.000      -6.840      -2.491
rank[T.1]      0.6754      0.316      2.134      0.033       0.055       1.296
rank[T.3]     -0.6648      0.283     -2.346      0.019      -1.220      -0.109
rank[T.4]     -0.8760      0.367     -2.389      0.017      -1.595      -0.157
gre            0.0023      0.001      2.070      0.038       0.000       0.004
gpa            0.8040      0.332      2.423      0.015       0.154       1.454
==============================================================================
"""

In [16]:
mylogit2.params

,0
Intercept,-4.665422
rank[T.1],0.675443
rank[T.3],-0.664761
rank[T.4],-0.876021
gre,0.002264
gpa,0.804038


In [17]:
mylogit2.params[0]['Intercept']

-4.665422001294611

In [18]:
# Option 3: calculate odds ratio R3/R2 based on data from mylogit
# using arighmetic in log-space

# From mylogit we have log odds
#  log(R2/R1) = -0.675443
#  log(R3/R1) = -1.340204

log_R2_over_R1 = mylogit.params[0]['rank[T.2]']
log_R3_over_R1 = mylogit.params[0]['rank[T.3]']

# We want
#  log(R3/R2)

# in the odds-space (ratios of probs) the  calculatin is
#    R3/R2 = (R3/R1) / (R2/R1) = R3_over_R1/R2_over_R1
# in log-space
#    log(R3/R2) = log(R3/R1) - log(R2/R1) = log_R2_over_R1 -log_R3_over_R1

log_R3_over_R2 = log_R3_over_R1 - log_R2_over_R1
log_R3_over_R2

-0.664760988504329

In [19]:
# check (by comparing to value obtained in mylogit2 where rank=2 is the reference)
np.isclose(log_R3_over_R2, mylogit2.params[0]['rank[T.3]'])

True

In [20]:
# Option 4: calculate odds ratio R3/R2 based on data from mylogit
# using arighmetic in probability space

# From mylogit we have log odds
#  log(R2/R1) = -0.675443
#  log(R3/R1) = -1.340204

R2_over_R1 = np.exp(mylogit.params[0]['rank[T.2]'])
R3_over_R1 = np.exp(mylogit.params[0]['rank[T.3]'])

# We want
#  R3/R2

# in the odds-space (ratios of probs) the  calculatin is
#    R3/R2 = (R3/R1) / (R2/R1) = R3_over_R1/R2_over_R1

R3_over_R2 = R3_over_R1/R2_over_R1
R3_over_R2

0.5143964596821655

In [21]:
# check (by comparing log of odds ratio to value obtained in mylogit2)
np.isclose(np.log(R3_over_R2), mylogit2.params[0]['rank[T.3]'])

True